# Examining the effect of including glacial runoff in SPEI drought risk calculations

Welcome!  This notebook will reproduce and visualise the analyses behind Ultee & Coats (manuscript title / submission status), using helper functions from gSPEI.py.  All code is stored in a public GitHub repository--click [here](https://github.com/ehultee/glacial-SPEI) for access.

### Loading in modules and data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from gSPEI import *

In [ ]:
fpath = 'Documents/GitHub/Data_unsynced/SPEI_files/' # local file path to SPEI output files

## Settings in filenames
integration_times = np.arange(3, 28, 4) # all SPEI integration times used
modelnames = ['CanESM2', 'CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GISS-E2-R', 'INMCM4', 'MIROC-ESM', 'NorESM1-M'] # all models used in comparison
scenarios = ['Rcp4p5', 'Rcp8p5'] # climate scenarios

## Basins in the order they are written in each file
basins = ['INDUS','TARIM','BRAHMAPUTRA','ARAL SEA','COPPER','GANGES','YUKON','ALSEK','SUSITNA','BALKHASH','STIKINE','SANTA CRUZ',
'FRASER','BAKER','YANGTZE','SALWEEN','COLUMBIA','ISSYK-KUL','AMAZON','COLORADO','TAKU','MACKENZIE','NASS','THJORSA','JOEKULSA A F.',
'KUSKOKWIM','RHONE','SKEENA','OB','OELFUSA','MEKONG','DANUBE','NELSON RIVER','PO','KAMCHATKA','RHINE','GLOMA','HUANG HE','INDIGIRKA',
'LULE','RAPEL','SANTA','SKAGIT','KUBAN','TITICACA','NUSHAGAK','BIOBIO','IRRAWADDY','NEGRO','MAJES','CLUTHA','DAULE/VINCES',
'KALIXAELVEN','MAGDALENA','DRAMSELV','COLVILLE']

Now we will load data into a dictionary for one specific climate scenario, indexed by model name.

In [ ]:
itime = integration_times[3] # select timescale of integration. [3] is 15 months, compatible with PDSI
scen = scenarios[0] # choose whether to load RCP 4.5 or RCP 8.5

SPEI_by_model = {m: {} for m in modelnames} # create dictionary indexed by model name
for m in modelnames:
    norunoff_f_m = fpath+'NRunoff_{}_{}_{}.txt'.format(itime, m, scen)
    wrunoff_f_m = fpath+'WRunoff_{}_{}_{}.txt'.format(itime, m, scen)
    SPEI_by_model[m]['NRunoff'] = np.loadtxt(norunoff_f_m)
    SPEI_by_model[m]['WRunoff'] = np.loadtxt(wrunoff_f_m)
    SPEI_by_model[m]['diff'] = SPEI_by_model[m]['WRunoff'] - SPEI_by_model[m]['NRunoff']

## Glacial effect per basin

We will use a helper function from gSPEI to plot the running mean glacial effect in a given basin.  Choose the basin by its index in the "basins" list.

In [ ]:
window = 30 # window in years over which to calculate the running mean
basin_id = 1 # index of basin of interest in "basins", above

plot_basin_runmean(basin_id=basin_id, permodel_dict=SPEI_by_model)

## Mean and variance of SPEI

When we account for glacial runoff in our SPEI calculation, how does the mean SPEI change?  Does SPEI become more or less variable?

We calculate the mean SPEI with and without glacial runoff, for each of our 8 climate models, over a 30-year period.  (By default, we do this at the end of the 21st century, but you can explore other periods by adjusting the "years" argument.)  We do the same for variance.  Then, we make an errorbar plot to visualize the inter-model range in each.

In [ ]:
## Calculate changes due to glacial effect at end of century, using gSPEI functions
yrs = (2070, 2100)

bas_glac_meanmed, mean_spread = glacial_meandiff(SPEI_by_model, years=yrs)
bas_glac_varmed, var_spread = glacial_vardiff(SPEI_by_model, years=yrs)

plt.figure('Mean and variance shifts due to glacial effects in 2070-2100')
plt.errorbar(x=bas_glac_meanmed, y=bas_glac_varmed, xerr=mean_spread, yerr=var_spread, ls='', marker='d', elinewidth=2.0, color='DarkBlue')
plt.axes().set_xlabel('Difference in mean SPEI', fontsize=16)
plt.axes().set_ylabel('Difference in SPEI variance', fontsize=16)
plt.axes().set_ylim(-1.5, 1.0)
plt.axes().set_xlim(-0.5, 5)
plt.show()
